In [2]:
import pandas as pd
import numpy as np

#this constant is the project planned cost. This is an actual from the database
project_cost = 1000



In [3]:
#ingestion 1
df_planned = pd.read_csv('Data/planned.csv',parse_dates=['Start_date','End_date'])
df_planned

,SubTaskID,SubTasks,Planned_Cost,Start_date,End_date
0,1,Activity 1,500,2026-01-03,2026-01-07
1,2,Activity 2,150,2026-01-08,2026-01-09
2,3,Activity 3,350,2026-01-10,2026-01-12


In [4]:
df_planned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   SubTaskID     3 non-null      int64         
 1   SubTasks      3 non-null      object        
 2   Planned_Cost  3 non-null      int64         
 3   Start_date    3 non-null      datetime64[ns]
 4   End_date      3 non-null      datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 252.0+ bytes


In [5]:
#computing for "Activity Weight" 
#formula for activity weight is subtask planned cost / project cost
#the computed values will be in a new column "Activity Weight"
df_planned['Activity_Weight'] = df_planned['Planned_Cost'] / project_cost
df_planned


,SubTaskID,SubTasks,Planned_Cost,Start_date,End_date,Activity_Weight
0,1,Activity 1,500,2026-01-03,2026-01-07,0.50
1,2,Activity 2,150,2026-01-08,2026-01-09,0.15
2,3,Activity 3,350,2026-01-10,2026-01-12,0.35


In [6]:
#adding durations in days
#adding holidays and weekends will affect this computation
df_planned['Duration_Days'] = (df_planned['End_date'] - df_planned['Start_date']).dt.days + 1
df_planned

,SubTaskID,SubTasks,Planned_Cost,Start_date,End_date,Activity_Weight,Duration_Days
0,1,Activity 1,500,2026-01-03,2026-01-07,0.50,5
1,2,Activity 2,150,2026-01-08,2026-01-09,0.15,2
2,3,Activity 3,350,2026-01-10,2026-01-12,0.35,3


In [7]:
#computing the average accomplishment per day
#formula is Activity weight / duration days
df_planned['Average_Accomplishement_per_Day'] = df_planned['Activity_Weight'] / df_planned['Duration_Days']
df_planned

,SubTaskID,SubTasks,Planned_Cost,Start_date,End_date,Activity_Weight,Duration_Days,Average_Accomplishement_per_Day
0,1,Activity 1,500,2026-01-03,2026-01-07,0.50,5,0.100000
1,2,Activity 2,150,2026-01-08,2026-01-09,0.15,2,0.075000
2,3,Activity 3,350,2026-01-10,2026-01-12,0.35,3,0.116667


In [8]:
#creating a shorter dataframe
df_short_planned = df_planned[['SubTaskID','SubTasks','Start_date','End_date','Average_Accomplishement_per_Day']]
df_short_planned

,SubTaskID,SubTasks,Start_date,End_date,Average_Accomplishement_per_Day
0,1,Activity 1,2026-01-03,2026-01-07,0.100000
1,2,Activity 2,2026-01-08,2026-01-09,0.075000
2,3,Activity 3,2026-01-10,2026-01-12,0.116667


In [9]:
#creating an empty dataframe
df_assembled = pd.DataFrame()

In [10]:
#assembling the dataframe

for index, row in df_short_planned.iterrows():
    df_container = pd.DataFrame() #temporary dataframe to hold generated dates and accomplishments
    print(f"Processing SubTaskID: {row['SubTaskID']}, SubTask: {row['SubTasks']}")
    print(f"{row['SubTasks']}: Start Date: {row['Start_date'].date()}, End Date: {row['End_date'].date()}, Average Accomplishment per Day: {row['Average_Accomplishement_per_Day']:.4f}")
    #generate dates between start and end dates for each activity
    activity_date_range = pd.date_range(start=row['Start_date'], end=row['End_date'], freq='D').tolist()
    df_container['Dates'] = activity_date_range
    df_container['SubTaskID'] = row['SubTaskID']
    df_container['SubTasks'] = row['SubTasks']
    df_container['Average_Accomplishment'] = row['Average_Accomplishement_per_Day']
    df_assembled = pd.concat([df_assembled, df_container], ignore_index=True)

#for notebook viewing only. Remove this in the actual .py file
df_assembled

Processing SubTaskID: 1, SubTask: Activity 1
Activity 1: Start Date: 2026-01-03, End Date: 2026-01-07, Average Accomplishment per Day: 0.1000
Processing SubTaskID: 2, SubTask: Activity 2
Activity 2: Start Date: 2026-01-08, End Date: 2026-01-09, Average Accomplishment per Day: 0.0750
Processing SubTaskID: 3, SubTask: Activity 3
Activity 3: Start Date: 2026-01-10, End Date: 2026-01-12, Average Accomplishment per Day: 0.1167


,Dates,SubTaskID,SubTasks,Average_Accomplishment
0,2026-01-03,1,Activity 1,0.100000
1,2026-01-04,1,Activity 1,0.100000
2,2026-01-05,1,Activity 1,0.100000
3,2026-01-06,1,Activity 1,0.100000
4,2026-01-07,1,Activity 1,0.100000
5,2026-01-08,2,Activity 2,0.075000
6,2026-01-09,2,Activity 2,0.075000
7,2026-01-10,3,Activity 3,0.116667
8,2026-01-11,3,Activity 3,0.116667
9,2026-01-12,3,Activity 3,0.116667


Integrating the bussiness days 

In [11]:
#Holidays
holidays_2026 = ['2026-01-01', '2026-02-25', '2026-04-09', '2026-05-01', '2026-06-12', '2026-08-31', '2026-11-30', '2026-12-25'] 
weekdays_active = "Mon Tue Wed Thu Fri Sat" #workdays including Saturdays

#creating the busdaycal object for convenient way of checking business days
busDays = np.busdaycalendar(weekmask=weekdays_active, holidays=holidays_2026)  

In [12]:
#business day array based on the 'Dates' column of df_assembled

#creating a numpy array of datetime64[D] type from the 'Dates' column
dates_np_array = df_assembled['Dates'].values.astype('datetime64[D]')

#creating the is business day array
busDayArray = np.is_busday(dates_np_array, busdaycal=busDays)

In [13]:
type(dates_np_array)

numpy.ndarray

In [14]:
busDayArray

array([ True, False,  True,  True,  True,  True,  True,  True, False,
        True])

In [15]:
#integrating the business day array into the dataframe

#create the new column 'Is a Business Day'
df_assembled['Is a Business Day'] = busDayArray
df_assembled

,Dates,SubTaskID,SubTasks,Average_Accomplishment,Is a Business Day
0,2026-01-03,1,Activity 1,0.100000,True
1,2026-01-04,1,Activity 1,0.100000,False
2,2026-01-05,1,Activity 1,0.100000,True
3,2026-01-06,1,Activity 1,0.100000,True
4,2026-01-07,1,Activity 1,0.100000,True
5,2026-01-08,2,Activity 2,0.075000,True
6,2026-01-09,2,Activity 2,0.075000,True
7,2026-01-10,3,Activity 3,0.116667,True
8,2026-01-11,3,Activity 3,0.116667,False
9,2026-01-12,3,Activity 3,0.116667,True


In [16]:
#rearraning the columns
df_assembled = df_assembled[[
    'Dates',
    'Is a Business Day',
    'SubTaskID',
    'SubTasks',
    'Average_Accomplishment'
]]

df_assembled

,Dates,Is a Business Day,SubTaskID,SubTasks,Average_Accomplishment
0,2026-01-03,True,1,Activity 1,0.100000
1,2026-01-04,False,1,Activity 1,0.100000
2,2026-01-05,True,1,Activity 1,0.100000
3,2026-01-06,True,1,Activity 1,0.100000
4,2026-01-07,True,1,Activity 1,0.100000
5,2026-01-08,True,2,Activity 2,0.075000
6,2026-01-09,True,2,Activity 2,0.075000
7,2026-01-10,True,3,Activity 3,0.116667
8,2026-01-11,False,3,Activity 3,0.116667
9,2026-01-12,True,3,Activity 3,0.116667


In [17]:
# creating a new column where instead of boolean business days values, it will be 1s and 0s

df_assembled['Is a Business Day_int'] = df_assembled['Is a Business Day'].astype(int)
df_assembled

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3244\2656052211.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assembled['Is a Business Day_int'] = df_assembled['Is a Business Day'].astype(int)


,Dates,Is a Business Day,SubTaskID,SubTasks,Average_Accomplishment,Is a Business Day_int
0,2026-01-03,True,1,Activity 1,0.100000,1
1,2026-01-04,False,1,Activity 1,0.100000,0
2,2026-01-05,True,1,Activity 1,0.100000,1
3,2026-01-06,True,1,Activity 1,0.100000,1
4,2026-01-07,True,1,Activity 1,0.100000,1
5,2026-01-08,True,2,Activity 2,0.075000,1
6,2026-01-09,True,2,Activity 2,0.075000,1
7,2026-01-10,True,3,Activity 3,0.116667,1
8,2026-01-11,False,3,Activity 3,0.116667,0
9,2026-01-12,True,3,Activity 3,0.116667,1


In [18]:
# Creating a new 'Average Accomplishment' columns where the accomplishment is only counted on business days(if business days is = 1)
df_assembled['Average_Accomplishment_Business_Days'] = df_assembled['Average_Accomplishment'] * df_assembled['Is a Business Day_int']
df_assembled

C:\Users\ASUS\AppData\Local\Temp\ipykernel_3244\288186325.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_assembled['Average_Accomplishment_Business_Days'] = df_assembled['Average_Accomplishment'] * df_assembled['Is a Business Day_int']


,Dates,Is a Business Day,SubTaskID,SubTasks,Average_Accomplishment,Is a Business Day_int,Average_Accomplishment_Business_Days
0,2026-01-03,True,1,Activity 1,0.100000,1,0.100000
1,2026-01-04,False,1,Activity 1,0.100000,0,0.000000
2,2026-01-05,True,1,Activity 1,0.100000,1,0.100000
3,2026-01-06,True,1,Activity 1,0.100000,1,0.100000
4,2026-01-07,True,1,Activity 1,0.100000,1,0.100000
5,2026-01-08,True,2,Activity 2,0.075000,1,0.075000
6,2026-01-09,True,2,Activity 2,0.075000,1,0.075000
7,2026-01-10,True,3,Activity 3,0.116667,1,0.116667
8,2026-01-11,False,3,Activity 3,0.116667,0,0.000000
9,2026-01-12,True,3,Activity 3,0.116667,1,0.116667


In [19]:
#dropping columns 'Average_Accomplishment' and 'Is a Business Day_int' as they are no longer needed
df_assembled = df_assembled.drop(columns=['Average_Accomplishment', 'Is a Business Day_int'])
df_assembled

,Dates,Is a Business Day,SubTaskID,SubTasks,Average_Accomplishment_Business_Days
0,2026-01-03,True,1,Activity 1,0.100000
1,2026-01-04,False,1,Activity 1,0.000000
2,2026-01-05,True,1,Activity 1,0.100000
3,2026-01-06,True,1,Activity 1,0.100000
4,2026-01-07,True,1,Activity 1,0.100000
5,2026-01-08,True,2,Activity 2,0.075000
6,2026-01-09,True,2,Activity 2,0.075000
7,2026-01-10,True,3,Activity 3,0.116667
8,2026-01-11,False,3,Activity 3,0.000000
9,2026-01-12,True,3,Activity 3,0.116667


In [20]:
#replacing zero values on column 'Average_Accomplishments_Business_Days' with NaN for better visualization in the s-curve graph
df_assembled['Average_Accomplishment_Business_Days'] = df_assembled['Average_Accomplishment_Business_Days'].replace(0, np.nan)
df_assembled

,Dates,Is a Business Day,SubTaskID,SubTasks,Average_Accomplishment_Business_Days
0,2026-01-03,True,1,Activity 1,0.100000
1,2026-01-04,False,1,Activity 1,NaN
2,2026-01-05,True,1,Activity 1,0.100000
3,2026-01-06,True,1,Activity 1,0.100000
4,2026-01-07,True,1,Activity 1,0.100000
5,2026-01-08,True,2,Activity 2,0.075000
6,2026-01-09,True,2,Activity 2,0.075000
7,2026-01-10,True,3,Activity 3,0.116667
8,2026-01-11,False,3,Activity 3,NaN
9,2026-01-12,True,3,Activity 3,0.116667


In [21]:
#exporting df_assembled dataframe to a csv file
df_assembled.to_csv('Output/assembled_data.csv', index=False)

In [22]:
#exporting df_assembled to a json file
df_assembled.to_json('Output/assembled_data.json', orient='records', date_format='iso')


In [23]:
#exporting df_assembled to ndjson file
df_assembled.to_json('Output/assembled_data.ndjson', orient='records', date_format='iso', lines=True)